In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd

def load_simulation_results(study_number, N_C, d, n):
    # Define the path to the results directory
    base_dir = f"/Users/jamiecuin/Documents/University/Imperial/MSc Statistics/Summer Project/EGM Changepoint Detection/Code/Simulation Studies/simulation_study_{study_number}/number_changepoints_{N_C}/number_nodes_{d}/number_samples_{n}/results/"
    
    # Paths to the CSV files
    sim_results_path = os.path.join(base_dir, "simulation_results.csv")
    changepoints_path = os.path.join(base_dir, "master_changepoint_locations.csv")
    
    # Load the CSV files into DataFrames
    sim_results_df = pd.read_csv(sim_results_path)
    changepoints_df = pd.read_csv(changepoints_path)
    
    # Merge the two DataFrames on the "simulation" and "permutation" columns
    merged_df = pd.merge(sim_results_df, changepoints_df, on=["simulation", "permutation"], how="inner")
    
    return merged_df

study_number = 101
N_C = 1
d = 20
n = 1000

merged_results = load_simulation_results(study_number, N_C, d, n)


In [ ]:
merged_results.head()

In [ ]:
def calculate_changepoint_evaluation(merged_df):
    """
    TODO
    """
    merged_df['changepoint_indices'] = merged_df['changepoint_indices'].astype(str)
    merged_df['predicted_changepoints'] = merged_df['predicted_changepoints'].astype(str)
    
    merged_df['true_changepoints_count'] = merged_df['changepoint_indices'].apply(lambda x: len(x.split(';')) if x != "nan" else 0)
    
    merged_df['predicted_changepoints_count'] = merged_df['predicted_changepoints'].apply(
        lambda x: len(x.split(';')) if x != "nan" else 0
    )
    
    merged_df['changepoints_diff'] = abs(merged_df['true_changepoints_count'] - merged_df['predicted_changepoints_count'])
    
    return merged_df

evaluation_results = calculate_changepoint_evaluation(merged_results)

In [ ]:
evaluation_results.head()

In [ ]:
evaluation_results['changepoints_diff'].mean()

In [ ]:
evaluation_results

In [ ]:
def change_points_to_segments(change_points, n):
    """
    TODO
    """
    segments = np.zeros(n, dtype=int)
    for i, cp in enumerate(change_points):
        segments[cp:] = i + 1
    return segments

def calculate_ari(estimated_cps, true_cps, n):
    """
    TODO
    """
    est_segments = change_points_to_segments(estimated_cps, n)
    true_segments = change_points_to_segments(true_cps, n)
    return adjusted_rand_score(true_segments, est_segments)

def calculate_average_ari(merged_df, n):
    """
    TODO
    """
    ari_scores = []
    
    for _, row in merged_df.iterrows():
        # Convert the true changepoints from strings to lists of integers
        true_cps = [int(x) for x in row['changepoint_indices'].split(';') if x and x != 'nan']
        
        # Handle NaN values or 'nan' strings in predicted changepoints
        if pd.isna(row['predicted_changepoints']) or row['predicted_changepoints'] == 'nan':
            estimated_cps = []
        else:
            estimated_cps = [int(float(x)) for x in row['predicted_changepoints'].split(';') if x and x != 'nan']
        
        # Calculate ARI for the current simulation
        ari = calculate_ari(estimated_cps, true_cps, n)
        ari_scores.append(ari)
    
    # Calculate and return the average ARI
    average_ari = np.mean(ari_scores)
    return average_ari

average_ari = calculate_average_ari(merged_results, n)

average_ari


In [ ]:
def change_points_to_segments(change_points, n):
    """
    TODO
    """
    segments = np.zeros(n, dtype=int)
    for i, cp in enumerate(change_points):
        segments[cp:] = i + 1
    return segments

def calculate_ari(estimated_cps, true_cps, n):
    """
    TODO
    """
    est_segments = change_points_to_segments(estimated_cps, n)
    true_segments = change_points_to_segments(true_cps, n)
    return adjusted_rand_score(true_segments, est_segments)

def calculate_ari_for_correct_count(merged_df, n):
    """
    TODO
    """
    ari_scores = []
    
    for _, row in merged_df.iterrows():
        true_cps = [int(x) for x in row['changepoint_indices'].split(';') if x and x != 'nan']
        
        if pd.isna(row['predicted_changepoints']) or row['predicted_changepoints'] == 'nan':
            estimated_cps = []
        else:
            estimated_cps = [int(float(x)) for x in row['predicted_changepoints'].split(';') if x and x != 'nan']
        
        if len(estimated_cps) == len(true_cps):
            ari = calculate_ari(estimated_cps, true_cps, n)
            ari_scores.append(ari)
    
    if ari_scores: 
        average_ari = np.mean(ari_scores)
    else:
        average_ari = np.nan  # Handle cases where no matches were found
    
    return average_ari

average_ari_for_correct_count = calculate_ari_for_correct_count(merged_results, n)

average_ari_for_correct_count
